# Определение уязвимых групп населения

Задача:
* кластеризовать регионы России и определить, какие из них наиболее
остро нуждаются в помощи малообеспеченным/неблагополучным
слоям населения;
* описать группы населения, сталкивающиеся с бедностью;
* определить:
    * влияет ли число детей, пенсионеров и других социально уязвимых
групп на уровень бедности в регионе;
    * связаны ли уровень бедности/социального неблагополучия с
производством и потреблением в регионе;
    * какие ещё зависимости можно наблюдать относительно
социально незащищённых слоёв населения.

In [1]:
import pandas as pd
import numpy as np

Для начала работы загрузим список субъектов РФ с сайта [Wikipedia](https://ru.wikipedia.org/wiki/Коды_субъектов_Российской_Федерации) и преобразуем их словарь.

In [2]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://ru.wikipedia.org/wiki/Коды_субъектов_Российской_Федерации"
regions = pd.read_html(requests.get(url).text, match='Наименование субъекта')[0]
regions['Наименование субъекта'] = regions['Наименование субъекта'].str.replace('\xa0', ' ')
regions = regions[regions['Наименование субъекта'] != 'Территории, находящиеся за пределами РФ и обслуживаемые Управлением режимных объектов МВД России, Байконур']
regions =regions.drop(['Код ГИБДД', 'Код ОКАТО и ОКТМО', 'Код ISO 3166-2[1]', 'Код ГОСТ 7.67-2003'], axis=1)
regions.head()

/var/folders/q_/7gwj_d0j19x6bt29_k4ngwl40000gq/T/ipykernel_51634/774025509.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  regions = pd.read_html(requests.get(url).text, match='Наименование субъекта')[0]


,Наименование субъекта
0,Республика Адыгея (Адыгея)
1,Республика Алтай
2,Республика Башкортостан
3,Республика Бурятия
4,Республика Дагестан


In [3]:
compl_regions = ['Ханты-Мансийский автономный округ — Югра', 'Ямало-Ненецкий автономный округ', 'Тюменская область', 'Ненецкий автономный округ', 'Архангельская область', 'Омская область']
normal_regions = regions[~regions['Наименование субъекта'].isin(compl_regions)]

normal_regions

,Наименование субъекта
0,Республика Адыгея (Адыгея)
1,Республика Алтай
2,Республика Башкортостан
3,Республика Бурятия
4,Республика Дагестан
...,...
82,Санкт-Петербург
83,Севастополь
84,Еврейская автономная область
87,Чукотский автономный округ


In [4]:
def short_regions(region):
    replace_dict = {'республика':'', 'автономная':'','край':'', 'область':'', 'автономный округ':'','народная':'', 'северная':'', 'автономная область':'', 'эл':'', '(':',', ')':'', ' — ':',',' - ':',',' ':''}
    reg_replace = {'саха': "республика саха", "алтай": "республика алтай"}
    for old, new in replace_dict.items():
        region = region.lower().replace(old, new)
            
    region = region.split(',')
    
    for key in reg_replace:
        if key in region:
            region.remove(key)
            region.append(reg_replace[key])
    return region

In [5]:
normal_regions['test'] = normal_regions['Наименование субъекта'].apply(short_regions)
normal_regions['test']

/var/folders/q_/7gwj_d0j19x6bt29_k4ngwl40000gq/T/ipykernel_51634/3188604570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_regions['test'] = normal_regions['Наименование субъекта'].apply(short_regions)


0       [адыгея, адыгея]
1     [республика алтай]
2         [башкортостан]
3              [бурятия]
4             [дагестан]
             ...        
82     [санкт-петербург]
83         [севастополь]
84           [еврейская]
87           [чукотский]
90         [харьковская]
Name: test, Length: 84, dtype: object

In [6]:
normal_regions.head()

,Наименование субъекта,test
0,Республика Адыгея (Адыгея),"[адыгея, адыгея]"
1,Республика Алтай,[республика алтай]
2,Республика Башкортостан,[башкортостан]
3,Республика Бурятия,[бурятия]
4,Республика Дагестан,[дагестан]


In [7]:
regions_dict = {}

for index, row in normal_regions.iterrows():
    for element in row['test']:
        regions_dict[element] = row['Наименование субъекта']
        
regions_dict

{'адыгея': 'Республика Адыгея (Адыгея)',
 'республика алтай': 'Республика Алтай',
 'башкортостан': 'Республика Башкортостан',
 'бурятия': 'Республика Бурятия',
 'дагестан': 'Республика Дагестан',
 'донецкая': 'Донецкая Народная Республика',
 'ингушетия': 'Республика Ингушетия',
 'кабардино-балкарская': 'Кабардино-Балкарская Республика',
 'калмыкия': 'Республика Калмыкия',
 'карачаево-черкесская': 'Карачаево-Черкесская Республика',
 'карелия': 'Республика Карелия',
 'коми': 'Республика Коми',
 'крым': 'Республика Крым',
 'луганская': 'Луганская Народная Республика',
 'марий': 'Республика Марий Эл',
 'мордовия': 'Республика Мордовия',
 'якутия': 'Республика Саха (Якутия)',
 'республика саха': 'Республика Саха (Якутия)',
 'осетия': 'Республика Северная Осетия — Алания',
 'алания': 'Республика Северная Осетия — Алания',
 'татарстан': 'Республика Татарстан (Татарстан)',
 'тыва': 'Республика Тыва',
 'удмуртская': 'Удмуртская Республика',
 'хакасия': 'Республика Хакасия',
 'чеченская': 'Чечен

In [10]:
# Функция для преобразования наименований субъектов:

def change_name(name):
    name = name.lower()
    
    for n in regions_dict:
        if n in name:
            return regions_dict[n]
    
    if re.search(r'\bомская\b(?!.*томская)', name):
        return "Омская область"
    
    if re.search(r'ханты', name):
        return 'Ханты-Мансийский автономный округ — Югра'
    elif re.search(r'югра', name):
        return 'Ханты-Мансийский автономный округ — Югра'
    elif re.search(r'ямало', name):
        return 'Ямало-Ненецкий автономный округ'
    elif 'тюменская' in name:
        return 'Тюменская область'
    
    if "ненецкий" in name and "архангельск" in name:
        return "Ненецкий автономный округ"
    elif re.search(r'\bненецкий\b', name) and not re.search(r'\bямало-ненецкий\b', name):
        return "Ненецкий автономный округ"
    elif "архангельск" in name:
        return "Архангельская область"
    
    print(f"No match found for '{name}'")
    return name

## 1. Обработка данных

### 1.1. Доходы населения

In [11]:
#Среднедущевые доходы населения

cash_capita = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx')

# Исключим строки содержащие федеральный округ, федерация и район
district = cash_capita[cash_capita['region'].str.contains('федеральный округ|Федерация|район|числе')].index
cash_capita.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
cash_capita['region'] = cash_capita['region'].apply(change_name)

cash_capita.set_index('region', inplace=True)

print('Среднедушевые денежные доходы (в месяц), руб')
cash_capita.head()

Среднедушевые денежные доходы (в месяц), руб


,2015,2016,2017,2018,2019,2020
region,,,,,,
Белгородская область,28043.0,29799.0,30342.0,30778.0,32352.0,32841.0
Брянская область,23428.0,24006.0,25107.0,26585.0,28371.0,28596.0
Владимирская область,22712.0,22365.0,23554.0,23539.0,25358.0,25922.0
Воронежская область,29366.0,29284.0,29498.0,30289.0,32022.0,32078.0
Ивановская область,22297.0,23676.0,24860.0,24503.0,25794.0,26277.0


In [12]:
duplicates_cash = cash_capita[cash_capita.index.duplicated(keep=False)]
duplicates_cash

,2015,2016,2017,2018,2019,2020
region,,,,,,
Архангельская область,31285.0,31394.0,32310.0,33831.0,35693.0,36779.0
Архангельская область,29716.0,29837.0,30707.0,32054.0,33874.0,34852.0
Тюменская область,41893.0,42657.0,44241.0,46124.0,48335.0,50059.0
Тюменская область,27448.0,27044.0,27672.0,29162.0,30662.0,31151.0


In [13]:
# Реальные доходы населения

real_incomes = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx', 1)

# Исключим строки содержащие федеральный округ, федерация и район
district = real_incomes[real_incomes['region'].str.contains('федеральный округ|Федерация|район|числе')].index
real_incomes.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
real_incomes['region'] = real_incomes['region'].apply(change_name)

real_incomes.set_index('region', inplace=True)

print('Реальные денежные доходы (в месяц), руб')
real_incomes.head()

Реальные денежные доходы (в месяц), руб


,2015,2016,2017,2018,2019,2020
region,,,,,,
Белгородская область,99.3,100.8,99.1,98.7,100.6,98.1
Брянская область,97.0,95.0,99.4,102.1,100.5,96.3
Владимирская область,99.5,92.2,100.9,96.4,101.9,98.0
Воронежская область,101.1,93.6,97.4,100.0,101.1,95.4
Ивановская область,95.5,98.5,100.4,94.4,99.5,97.2


In [14]:
# Номинальная заработная плата

formal_wage_paid = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx', 2)

# Исключим строки содержащие федеральный округ, федерация и район
district = formal_wage_paid[formal_wage_paid['region'].str.contains('федеральный округ|Федерация|район|числе')].index
formal_wage_paid.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
formal_wage_paid['region'] = formal_wage_paid['region'].apply(change_name)

formal_wage_paid.set_index('region', inplace=True)

print('Номинальная заработная плата (в месяц), руб')
formal_wage_paid.head()

Номинальная заработная плата (в месяц), руб


,2015,2016,2017,2018,2019,2020
region,,,,,,
Белгородская область,25456.0,27091.0,29066.0,31852.0,34615.0,37442.0
Брянская область,21679.0,22923.0,24743.0,27251.0,29853.0,31946.0
Владимирская область,23877.0,25135.0,26975.0,30460.0,33076.0,35240.0
Воронежская область,24906.0,26335.0,28007.0,31207.0,33690.0,36317.0
Ивановская область,21161.0,22144.0,23470.0,25729.0,27553.0,29083.0


In [15]:
# Реальная заработная плата

real_pay = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx', 2)

# Исключим строки содержащие федеральный округ, федерация и район
district = real_pay[real_pay['region'].str.contains('федеральный округ|Федерация|район|числе')].index
real_pay.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
real_pay['region'] = real_pay['region'].apply(change_name)

real_pay.set_index('region', inplace=True)

print('Реальная заработная плата (в месяц), руб')
real_pay.head()

Реальная заработная плата (в месяц), руб


,2015,2016,2017,2018,2019,2020
region,,,,,,
Белгородская область,25456.0,27091.0,29066.0,31852.0,34615.0,37442.0
Брянская область,21679.0,22923.0,24743.0,27251.0,29853.0,31946.0
Владимирская область,23877.0,25135.0,26975.0,30460.0,33076.0,35240.0
Воронежская область,24906.0,26335.0,28007.0,31207.0,33690.0,36317.0
Ивановская область,21161.0,22144.0,23470.0,25729.0,27553.0,29083.0


### 1.2. Число умерших на первом году жизни детей за год

In [16]:
# Детская смертность в сельской местности

rural = pd.read_excel('data/child_mortality_rural_1990_2021.xls', skiprows=2)

# Удалим лишние столбцы
rural.drop(['Unnamed: 1', 'Unnamed: 2'], axis = 1, inplace=True)

# Исключим строки содержащие федеральный округ, федерация и район
district = rural[rural['Unnamed: 0'].str.contains('федеральный округ|Федерация|район')].index
# Удаляем пробелы перед названием регионов
rural['Unnamed: 0'] = rural['Unnamed: 0'].str.strip()
rural.drop(district, axis = 0, inplace=True)

rural.rename(columns={'Unnamed: 0':"region"}, inplace=True)

# Унифицируем названия регионов
rural['region'] = rural['region'].apply(change_name)

rural.set_index('region', inplace=True)

No match found for 'усть-ордынский бурятский округ'
No match found for 'корякский округ, входящий в состав камчатского края'


In [17]:
# Детская смертность в городской местности

urban = pd.read_excel('data/child_mortality_urban_1990_2021.xls', skiprows=2)

# Удалим лишние столбцы
urban.drop(['Unnamed: 1', 'Unnamed: 2'], axis = 1, inplace=True)

# Исключим строки содержащие федеральный округ, федерация и район
district = urban[urban['Unnamed: 0'].str.contains('федеральный округ|Федерация|район')].index
# Удаляем пробелы перед названием регионов
urban['Unnamed: 0'] = urban['Unnamed: 0'].str.strip()
urban.drop(district, axis = 0, inplace=True)

urban.rename(columns={'Unnamed: 0':"region"}, inplace=True)

# Унифицируем названия регионов
urban['region'] = urban['region'].apply(change_name)

urban.set_index('region', inplace=True)

# Объединяем детскую смертность с городов и регионов
child_mortality = rural.add(urban)
print('Количетсво умерших детей, чел.')
child_mortality.head()

No match found for 'усть-ордынский бурятский округ'
No match found for 'корякский округ, входящий в состав камчатского края'
Количетсво умерших детей, чел.


,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Белгородская область,312.0,290.0,240.0,244.0,233.0,203.0,174.0,167.0,160.0,161.0,...,127.0,116.0,103.0,110.0,108.0,77.0,73.0,39.0,47.0,54.0
Брянская область,322.0,304.0,283.0,297.0,256.0,229.0,212.0,202.0,182.0,184.0,...,127.0,124.0,126.0,111.0,103.0,96.0,45.0,37.0,41.0,27.0
Владимирская область,301.0,267.0,239.0,210.0,211.0,196.0,161.0,161.0,169.0,174.0,...,127.0,117.0,110.0,107.0,100.0,76.0,67.0,58.0,61.0,66.0
Воронежская область,399.0,441.0,418.0,362.0,348.0,321.0,273.0,275.0,302.0,280.0,...,166.0,186.0,145.0,126.0,120.0,108.0,99.0,81.0,70.0,83.0
Ивановская область,255.0,231.0,214.0,225.0,192.0,182.0,159.0,166.0,137.0,160.0,...,71.0,86.0,68.0,64.0,66.0,35.0,35.0,40.0,28.0,24.0


### 1.3 Люди с инвалидностью

In [18]:
disabled = pd.read_csv('data/disabled_total_by_age_2017_2022.csv')

# Исключаем строки содержащие федеральный округ, федерация и район
district = disabled[disabled['region'].str.contains('федеральный округ|Федерация|район|Байконур')].index
disabled.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
disabled['region'] = disabled['region'].apply(change_name)

disabled.set_index('region', inplace=True)

# Выполняем группировку по году
disabled['date'] = pd.to_datetime(disabled['date']).dt.year
disabled = disabled.groupby(['region','date'])['total'].sum().unstack()

print('Количество инвалидов, чел.')
disabled.head()

Количество инвалидов, чел.


date,2017,2018,2019,2020,2021,2022
region,,,,,,
Алтайский край,1987350.0,2120203.0,2100077.0,2060817.0,1986766.0,646296.0
Амурская область,816315.0,797050.0,785693.0,766768.0,733732.0,236444.0
Архангельская область,1056888.0,1038542.0,1031766.0,1016584.0,978993.0,316492.0
Астраханская область,533426.0,525601.0,523200.0,520687.0,511039.0,167664.0
Белгородская область,2625755.0,2522995.0,2446166.0,2359129.0,2252362.0,723760.0


### 1.4 Сведения о заболеваемости алкоголизмом и наркоманией

In [19]:
alco = pd.read_excel('data/drug_alco.xlsx', 0)
alco1718 = pd.read_excel('data/drug_alco.xlsx', 1)

# Исключаем строки содержащие федеральный округ, федерация и район
district = alco[alco['region'].str.contains('федеральный округ|Федерация|район|Байконур')].index
alco.drop(district, axis = 0, inplace=True)
district = alco1718[alco1718['region'].str.contains('федеральный округ|Федерация|район|Байконур')].index
alco1718.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
alco['region'] = alco['region'].apply(change_name)
alco1718['region'] = alco1718['region'].apply(change_name)

alco.set_index('region', inplace=True)
alco1718.set_index('region', inplace=True)

# alco = pd.concat([alco, alco1718], axis=1).sort_index()

# print('Количество человек с алкогольной зависимостью поставленных на учёт на 100 тысяч человек')
# display(alco.head())

No match found for 'город санкт - петербург'
No match found for 'главное медицинское управление управления делами президента российской федерации'


In [21]:
drugs = pd.read_excel('data/drug_alco.xlsx', 2)
drugs1718 = pd.read_excel('data/drug_alco.xlsx', 3)

# Исключаем строки содержащие федеральный округ, федерация и район
district = drugs[drugs['region'].str.contains('федеральный округ|Федерация|район|Байконур')].index
drugs.drop(district, axis = 0, inplace=True)
district = drugs1718[drugs1718['region'].str.contains('федеральный округ|Федерация|район|Байконур')].index
drugs1718.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
drugs['region'] = drugs['region'].apply(change_name)
drugs1718['region'] = drugs1718['region'].apply(change_name)

drugs.set_index('region', inplace=True)
drugs1718.set_index('region', inplace=True)

drugs = pd.concat([drugs, drugs1718], axis=1).sort_index()

print('Количество человек с наркотической зависимостью поставленных на учёт на 100 тысяч человек')
drugs.head()

No match found for 'город санкт - петербург'
No match found for 'главное медицинское управление управления делами президента российской федерации'
Количество человек с наркотической зависимостью поставленных на учёт на 100 тысяч человек


,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,29.8,35.2,35.2,37.7,27.3,24.5,26.4,33.4,22.9,21.6,20.7,17.4,17.4,17.4
Амурская область,21.3,27.2,27.2,28.4,26.2,37.1,33.9,24.2,22.5,22.2,19.1,18.9,18.7,24.4
Архангельская область,1.5,1.4,1.4,2.6,3.4,4.6,6.4,4.0,4.0,NaN,NaN,NaN,5.4,9.2
Астраханская область,10.7,12.9,12.9,6.8,8.1,9.8,7.5,5.1,2.3,NaN,NaN,NaN,0.7,0.2
Белгородская область,4.5,9.5,9.5,4.2,3.9,4.8,4.0,2.3,2.4,1.9,2.0,1.6,2.3,3.2


### 1.5 Валовой региональный продукт на душу населения, в рублях.

In [22]:
grass = pd.read_excel('data/gross_regional_product_1996_2020.xls', 
                      header=2)

# Исключим строки содержащие федеральный округ, федерация и район
district = grass[grass['Unnamed: 0'].str.contains('федеральный округ|Федерация|район')].index
# Удаляем пробелы перед названием регионов
grass['Unnamed: 0'] = grass['Unnamed: 0'].str.strip()
grass.drop(district, axis = 0, inplace=True)

grass.rename(columns={'Unnamed: 0':"region"}, inplace=True)

# Унифицируем названия регионов
grass['region'] = grass['region'].apply(change_name)

grass.set_index('region', inplace=True)

print('Валовой региональный продукт на душу населения, в рублях.')
grass.head()

Валовой региональный продуктна душу населения, в рублях.


,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,,,,,,,
Белгородская область,9575.6,10792.2,12242.8,21398.0,27969.5,33126.7,41327.4,50271.4,75629.4,95911.2,...,331010.0,354570.6,368874.8,400820.8,447619.7,501467.8,539720.5,588641.5,617426.5,646569.0
Брянская область,7275.3,7175.7,7659.1,11752.4,17413.5,21511.9,27020.0,31953.4,37719.1,49923.4,...,137187.1,164726.6,175865.0,196096.7,221080.0,258752.7,280630.1,304547.1,332442.8,347204.5
Владимирская область,7620.7,8431.3,9350.2,15457.1,21073.3,27170.0,32923.6,40809.4,49353.4,58261.0,...,181842.6,200456.4,216320.8,232757.6,262945.5,309713.7,325043.1,349856.4,394560.3,410443.6
Воронежская область,7651.9,8686.6,9082.1,14808.3,20365.1,24905.4,34789.6,42237.5,49530.0,56534.5,...,203575.5,241947.4,262578.3,308004.7,345566.8,354657.9,374125.7,408140.7,431037.0,459629.5
Ивановская область,6725.4,6379.3,6804.5,9765.2,14240.0,18947.2,23396.9,29192.4,35732.7,40039.1,...,121945.5,129448.3,151263.6,146032.6,174687.5,200504.4,208522.0,230325.5,249591.6,273821.5


### 1.6 Оценка домохозяйствами состояния занимаемого ими жилого помещения

In [23]:
housing = pd.read_excel('data/housing_2020.xlsx')

housing.rename(columns={'Регион':"region"}, inplace=True)

# Исключим строки содержащие федеральный округ, федерация и район
district = housing[housing['region'].str.contains('федеральный округ|Федерация|район')].index
housing.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
housing['region'] = housing['region'].apply(change_name)

housing.set_index('region', inplace=True)

housing_conditions = pd.read_excel('data/housing_2020.xlsx', 1)
housing_conditions.rename(columns={'Регион':"region"}, inplace=True)

housing_conditions = housing_conditions.dropna(subset=['region'])

# Исключим строки содержащие федеральный округ, федерация и район
district = housing_conditions[housing_conditions['region'].str.contains('федеральный округ|Федерация|район')].index
housing_conditions.drop(district, axis = 0, inplace=True)


# Унифицируем названия регионов
housing_conditions['region'] = housing_conditions['region'].apply(change_name)

housing_conditions.set_index('region', inplace=True)

print('Средняя жилая площадь на человека, м^2')
living_area = housing[['Размер жилой площади в расчете на члена домохозяйства']]
display(living_area.head())

print('Плохие условия проживания')
# ВЫбираем столбец с плохими условиями
poor_living_conditions = housing_conditions[['из них указавшие: на плохое состояние или очень плохое состояние жилого помещения и на стесненность проживания']]
# Удалаям последнюю строку
poor_living_conditions = poor_living_conditions.drop(poor_living_conditions.index[-1])
poor_living_conditions.rename(columns={'из них указавшие: на плохое состояние или очень плохое состояние жилого помещения и на стесненность проживания':'poor_living'}, inplace=True)
display(poor_living_conditions.head())

No match found for 'bладимирская область'
No match found for 'bоронежская область'
No match found for 'bологодская область'
No match found for 'bолгоградская область'
No match found for 'bладимирская область'
No match found for 'bоронежская область'
No match found for 'bологодская область'
No match found for 'bолгоградская область'
No match found for '* здесь и далее - отметка " … " (многоточие) означает, что число ответов респондентов (число наблюдений) по данному показателю составляет менее 50'
Средняя жилая площадь на человека, м^2


,Размер жилой площади в расчете на члена домохозяйства
region,
Белгородская область,17.8
Брянская область,24.1
bладимирская область,16.1
bоронежская область,19.9
Ивановская область,18.1


Плохие условия проживания


,poor_living
region,
Белгородская область,0.4
Брянская область,0.0
bладимирская область,0.0
bоронежская область,0.1
Ивановская область,0.9


### 1.7 Заболеваемость

In [30]:
diseases = pd.read_excel('data/morbidity_2005_2020_age_disease.xls', skiprows=2)
diseases.rename(columns={'Unnamed: 0':"region",'Unnamed: 1':"Заболевания",'Unnamed: 2':"Группа"}, inplace=True)


# Исключим строки содержащие федеральный округ, федерация и район
district = diseases[diseases['region'].str.contains('федеральный округ|Федерация|район')].index
# Удаляем пробелы перед названием регионов
diseases['region'] = diseases['region'].str.strip()
diseases.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
diseases['region'] = diseases['region'].apply(change_name)

# Назначим регион как индекс
diseases.set_index('region', inplace=True)
diseases = diseases[diseases['Группа'] == 'Всего']

# Выделим статистику по общем заболиваемости
general_morbidity = diseases[diseases['Заболевания'].isin(['Все заболевания'])].loc[:,['2015', '2016','Заболевания']]
general_morbidity = general_morbidity.dropna(axis=0)
general_morbidity = general_morbidity.groupby(['region','Заболевания'])[['2015', '2016']].sum().unstack()
print('Статистика по общей заболиваемости')
general_morbidity.head()

Статистика по общей заболиваемости


,2015,2016
Заболевания,Все заболевания,Все заболевания
region,,
Алтайский край,109652.6,112764.7
Амурская область,83919.0,83412.0
Архангельская область,101587.4,99842.1
Астраханская область,62055.8,64679.9
Белгородская область,70736.1,74637.4


### 1.8 Подростковая беременность

In [31]:
diseases = pd.read_excel('data/morbidity_2005_2020_age_disease.xls', skiprows=2)
diseases.rename(columns={'Unnamed: 0':"region",'Unnamed: 1':"Заболевания",'Unnamed: 2':"Группа"}, inplace=True)

# Исключим строки содержащие федеральный округ, федерация и район
district = diseases[diseases['region'].str.contains('федеральный округ|Федерация|район')].index
# Удаляем пробелы перед названием регионов
diseases['region'] = diseases['region'].str.strip()
diseases.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
diseases['region'] = diseases['region'].apply(change_name)

# Назначим регион как индекс
diseases.set_index('region', inplace=True)
pregnancy = diseases[diseases['Заболевания'] == 'Беременность, роды и послеродовой период']

# Объеденим значения с таблиц по беременности детей до 14 и до 17 лет.
pregnancy_14 = pregnancy[pregnancy['Группа'] == '0-14 лет'].copy()
pregnancy_17 = pregnancy[pregnancy['Группа'] == '15-17 лет'].copy()

pregnancy_14.drop(['Группа', 'Заболевания'], axis = 1, inplace=True)
pregnancy_17.drop(['Группа', 'Заболевания'], axis = 1, inplace=True)
teen_pregnancy = pregnancy_14.add(pregnancy_17, fill_value=0)

# Удалим статистику до 2014 года, до 2014 года статистика велаась без автономных округов
teen_pregnancy.drop(['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013'], axis = 1, inplace=True)
print('Подростковая беременность, чел. на 100 тыс. человек')
display(teen_pregnancy.head())

Подростковая беременность, чел. на 100 тыс. человек


,2014,2015,2016
region,,,
Белгородская область,1264.7,1227.3,875.6
Брянская область,1076.7,862.7,705.5
Владимирская область,1771.9,1442.4,856.4
Воронежская область,553.1,369.8,272.0
Ивановская область,1023.5,542.5,491.3


### 1.9 Рождаемость

In [33]:
import regex as re

standard_names = regions_dict.copy()

newborn = pd.read_csv(
    'data/newborn_2006_2022_monthly.csv', sep=';', decimal=',')
newborn.rename({'Region': 'region'}, axis=1, inplace=True)
names_to_drop = [
    'Архангельская область (кроме Ненецкого автономного округа)',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    'Московская обл. в старых границах',
    'Москва в старых границах'
]

# Исключим строки содержащие федеральный округ, федерация и район
district = newborn[newborn['region'].str.contains('федеральный округ|Федерация|район')].index
newborn.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
newborn['region'] = newborn['region'].apply(change_name)

# Назначим регион как индекс
newborn.set_index('region', inplace=True)
    
# Суммируем данные за год
for year in range(2006, 2023):
    cols = []
    for col in newborn.columns:
        if str(year) in str(col):
            cols.append(col)
    newborn[year] = newborn[cols].sum(axis=1)
    newborn.drop(columns=cols, inplace=True)
newborn.drop(columns=2022, inplace=True)  # Неполный год


newborn.head()

No match found for 'усть-ордынский бурятский округ'
No match found for 'корякский округ, входящий в состав камчатского края'


,Unnamed: 198,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,
Белгородская область,NaN,14269.0,15666.00,16810.00,16911.00,16669.00,16894.00,17913.00,17883.00,17822.00,17864.00,17261.0,15113.00,14318.00,13197.99,12351.99,12223.0
Брянская область,NaN,12042.0,13282.99,14319.00,14406.00,13773.00,13806.00,14376.00,13855.00,13657.99,14067.00,13379.0,11552.99,11124.00,10028.00,9480.00,9047.0
Владимирская область,NaN,13915.0,14679.00,15699.00,15520.00,15540.00,15644.00,16445.00,15777.00,15817.00,16284.00,15662.0,13395.00,12658.00,11158.00,10120.00,9780.0
Воронежская область,NaN,19208.0,20568.99,22421.99,23560.99,23819.99,23826.97,25373.99,24980.97,25331.99,25923.98,24938.0,22423.00,21381.99,19643.00,19161.00,18593.0
Ивановская область,NaN,9818.0,10617.00,11138.00,11329.98,11131.00,11035.00,11581.00,11769.00,11694.97,11761.00,11184.0,9877.00,9134.00,7951.00,7563.00,7327.0


### 1.10 процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам

In [34]:
poverty_percent = pd.read_csv('data/poverty_percent_by_regions_1992_2020.csv')
poverty_percent = pd.pivot_table(data=poverty_percent, 
        values='poverty_percent', columns='year', index='region').reset_index()

# Исключим строки содержащие федеральный округ, федерация и район
district = poverty_percent[poverty_percent['region'].str.contains('федеральный округ|Федерация|район')].index
# Удаляем пробелы перед названием регионов
poverty_percent['region'] = poverty_percent['region'].str.strip()
poverty_percent.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
poverty_percent['region'] = poverty_percent['region'].apply(change_name)

# Назначим регион как индекс
poverty_percent.set_index('region', inplace=True)

poverty_percent.head(3)

year,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,,,,,,,
Архангельская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.0,14.5,14.5,16.2,14.9,13.9,12.5,12.7,12.3
Ненецкий автономный округ,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,37.9,26.9,...,7.7,6.5,7.8,8.9,9.4,10.1,10.6,9.7,9.4,9.4
Ханты-Мансийский автономный округ — Югра,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.4,11.8,12.0,14.7,15.5,15.4,14.3,14.4,13.6


### 1.11 Распределение малоимущего населения по социально-демографическим группам

In [35]:
def socdem(x):
    """Изменяем название столбца с регионами, приводим название регионов
    к общему виду.

    Args:
        x (xls): таблица с данными
    Returns:
        DataFrame: изменнённый датафрейм
    """
    df = pd.read_excel(x, header=2)
    df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
    district = df[df['region'].str.contains('федеральный округ|Федерация|район')].index
    # Удаляем пробелы перед названием регионов
    df['region'] = df['region'].str.strip()
    df.drop(district, axis = 0, inplace=True)
    df['region'] = df['region'].apply(change_name)
    df.set_index('region', inplace=True)
    return df

poverty_socdem_2017 = socdem(
    'data/poverty_socdem_2017.xls')
poverty_socdem_2018 = socdem(
    'data/poverty_socdem_2018.xls')
poverty_socdem_2019 = socdem(
    'data/poverty_socdem_2019.xls')
poverty_socdem_2020 = socdem(
    'data/poverty_socdem_2020.xls')

# Приведём всё к одному датафрейму, 
#заменим пропуски и нули значениями из прошлый таблиц
proverty_socdem = poverty_socdem_2020.replace({0: np.nan}).combine_first(poverty_socdem_2019)
proverty_socdem = proverty_socdem.replace({0:np.nan}).combine_first(poverty_socdem_2018)
proverty_socdem = proverty_socdem.replace({0:np.nan}).combine_first(poverty_socdem_2017)

print('Процентное соотношение малоимущего населения по группам')
proverty_socdem.head()

Процентное соотношение малоимущего населения по группам


,Все население,Дети в возрасте до 16 лет,Население старше трудоспособного возраста,Население трудоспособного возраста
region,,,,
Алтайский край,100,31.4,9.2,59.4
Амурская область,100,38.4,4.7,56.9
Архангельская область,100,30.2,6.7,63.1
Архангельская область,100,30.2,6.7,63.1
Архангельская область,100,28.5,7.2,64.3


### 1.12 Объём отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами.

In [36]:
reg_prod = pd.read_csv('data/regional_production_2017_2020.csv')

# Исключим строки содержащие федеральный округ, федерация и район
district = reg_prod[reg_prod['region'].str.contains('федеральный округ|Федерация|район')].index
# Удаляем пробелы перед названием регионов
reg_prod['region'] = reg_prod['region'].str.strip()
reg_prod.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
reg_prod['region'] = reg_prod['region'].apply(change_name)

# Назначим регион как индекс
reg_prod.set_index('region', inplace=True)

# Выделим датафрейм с показателем по добыче полезных ископаемых
reg_mining = reg_prod[reg_prod['production_field'] == 'ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ'].loc[:,'2017':'2020']
# Выделим датафрейм с показателем по обеспечения электроэнергией
reg_electric = reg_prod[reg_prod['production_field'] == 'ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА'].loc[:,'2017':'2020']
# Выделим показатель водоснабжения, утилизации отходов и ликвидации загрязнений
reg_water = reg_prod[reg_prod['production_field'] == 'ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ'].loc[:,'2017':'2020']
# Выделим датафрейм с показателем по обрабатывающим производствам
reg_manuf = reg_prod[reg_prod['production_field'] == 'ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА'].loc[:,'2017':'2020']
# Выделим датафрейм с показателем по промышленности
reg_indust = reg_prod[reg_prod['production_field'] == 'Промышленное производство (промышленность)'].loc[:,'2017':'2020']

print('Добыча полезных ископаемых, в тысячах рублей: ')
display(reg_mining.head(2))
print('ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА, в тысячах рублей:')
display(reg_electric.head(2))
print('ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ, в тысячах рублей:')
display(reg_water.head(2))
print('ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА, в тысячах рублей:')
display(reg_manuf.head(2))
print('Промышленное производство (промышленность), в тысячах рублей:')
display(reg_indust.head(2))

Добыча полезных ископаемых, в тысячах рублей: 


,2017,2018,2019,2020
region,,,,
Белгородская область,116588313.5,148863422.7,178389894.9,189803813.7
Брянская область,277427.7,261549.5,338027.3,310665.9


ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА, в тысячах рублей:


,2017,2018,2019,2020
region,,,,
Белгородская область,27030715.9,26805485.3,26542583.5,26787381.9
Брянская область,16064108.9,17730258.7,17708749.5,17876399.5


ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ, в тысячах рублей:


,2017,2018,2019,2020
region,,,,
Белгородская область,10614103.8,8936415.0,11404169.8,11175025.3
Брянская область,7347143.3,8901072.7,9258735.3,9588966.4


ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА, в тысячах рублей:


,2017,2018,2019,2020
region,,,,
Белгородская область,629092604.8,7.108288e+08,7.226997e+08,7.463807e+08
Брянская область,191018941.6,2.185441e+08,2.536616e+08,2.556862e+08


Промышленное производство (промышленность), в тысячах рублей:


,2017,2018,2019,2020
region,,,,
Белгородская область,783325738.0,8.954341e+08,9.390364e+08,9.741469e+08
Брянская область,214707621.5,2.454370e+08,2.809671e+08,2.834622e+08


### 1.13 Оборот розничной торговли на душу населения, в рублях.

In [37]:
retail = pd.read_excel('data/retail_turnover_per_capita_2000_2021.xls', 
                        header=2, usecols='A, P:X')
retail.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
retail.drop(0, inplace=True)

# Исключим строки содержащие федеральный округ, федерация и район
district = retail[retail['region'].str.contains('федеральный округ|Федерация|район')].index
# Удаляем пробелы перед названием регионов
retail['region'] = retail['region'].str.strip()
retail.drop(district, axis = 0, inplace=True)

# Унифицируем названия регионов
retail['region'] = retail['region'].apply(change_name)

# Назначим регион как индекс
retail.set_index('region', inplace=True)

retail.head()

No match found for 'усть-ордынский бурятский округ'
No match found for 'корякский округ, входящий в состав камчатского края'


,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,
Белгородская область,144992,164079,178097,192504,203298,217059,230704,237076,259151
Брянская область,138669,158501,177048,179767,192775,209987,225818,225643,257679
Владимирская область,116202,129564,139408,143643,153088,164519,175251,177680,207670
Воронежская область,158218,181499,198814,208638,221302,236953,251882,252416,296665
Ивановская область,121813,138185,143349,143568,153993,163290,178517,180314,211546
